In [ ]:
!pip install blocksnet

In [ ]:
import osmnx as ox
import numpy as np
import geopandas as gpd
import pandas as pd
import blocksnet as bn
import os
from blocksnet import AdjacencyCalculator

In [ ]:
# Загружаем датасет трафика и граф ВО
#dataset_speeds = np.load('/content/traffic_9_10_11_month_work_hours.npz')['arr_0'] # в arr_0 хранится скорость, arr_1 - время, arr_2 - дата, arr_3 - день недели
dataset_speeds = np.load('/content/traffic_prediction_last_45_hours_ATDM.npz')['arr_0']
G = ox.load_graphml("/content/drive.graphml")

In [ ]:
# Формирование дорожного графа
def get_new_graph(speed, graph, path_to_save):
    node, edges = ox.graph_to_gdfs(graph)
    edges['speed'] = speed
    for i in range(len(edges)):
        edges['weight'].iloc[i] = edges['length'].iloc[i]/(edges['speed'].iloc[i]*(1000/60))
    new_graph = ox.graph_from_gdfs(node, edges)
    ox.save_graphml(new_graph, path_to_save)

# Формирование серии графов в течении дня
def get_road_graphs_during_day(speeds_dataset, graph, path_folder):
    for hour in range(len(speeds_dataset)):
        speed = speeds_dataset[hour]
        path_to_save_file = f'{path_folder}/road_graph_{hour}.graphml'
        get_new_graph(speed, graph, path_to_save_file)

In [ ]:
get_road_graphs_during_day(dataset_speeds, G, '/content/predict_traffic')

In [ ]:
blocks = gpd.read_parquet('/content/blocks_cutter_result.parquet')

def weight_to_float(graph):
  for u, v, data in graph.edges(data=True):
    data['weight'] = float(data['weight'])

In [ ]:
for hour in range(45):
  name_file = f'/content/predict_traffic/road_graph_{hour}.graphml'
  name = f'ac_matrix_{hour}.pickle'
  g = ox.load_graphml(name_file)
  weight_to_float(g)
  accessibility_matrix = AdjacencyCalculator(blocks=blocks, graph=g).get_dataframe()
  accessibility_matrix.to_pickle(os.path.join('/content/predict_traffic/', name))

In [ ]:
from google.colab import files
for hour in range(5):
  files.download(f'/content/predict_traffic/ac_matrix_4{hour}.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Формирование дорожного графа
def get_new_graph(speed, graph, path_to_save):
    node, edges = ox.graph_to_gdfs(graph)
    edges['speed'] = speed
    for i in range(len(edges)):
        edges['weight'].iloc[i] = edges['length'].iloc[i]/(edges['speed'].iloc[i]*(1000/60))
    new_graph = ox.graph_from_gdfs(node, edges)
    ox.save_graphml(new_graph, path_to_save)

# Формирование серии графов в течении дня
def get_road_graphs_during_day(day, hours_per_day, speeds_dataset, graph, path_folder):
    for hour in range(hours_per_day):
        speed = speeds_dataset[day*hours_per_day+hour]
        path_to_save_file = f'{path_folder}/road_graph_{day}_{hour}.graphml'
        get_new_graph(speed, graph, path_to_save_file)


In [ ]:
# Получим графы 21.09.2023 (четверг)
day = 20
hours = 15
date = np.load('/content/traffic_9_10_11_month_work_hours.npz')['arr_2']
week_day = np.load('/content/traffic_9_10_11_month_work_hours.npz')['arr_3']
print(date[day*hours] + ' ' + week_day[day*hours])

get_road_graphs_during_day(day, hours, dataset_speeds, G, '/content/graphml')

In [ ]:
blocks = gpd.read_parquet('/content/blocks_cutter_result.parquet')

def weight_to_float(graph):
  for u, v, data in graph.edges(data=True):
    data['weight'] = float(data['weight'])

In [ ]:
for hour in range(hours):
  name_file = f'/content/graphml/road_graph_{day}_{hour}.graphml'
  name = f'ac_matrix_{day}_{hour}.pickle'
  g = ox.load_graphml(name_file)
  weight_to_float(g)
  accessibility_matrix = AdjacencyCalculator(blocks=blocks, graph=g).get_dataframe()
  accessibility_matrix.to_pickle(os.path.join('/content/ac_matrix/', name))

In [ ]:
accessibility_matrix = pd.read_pickle("/content/ac_matrix/ac_matrix_20_0.pickle")
accessibility_matrix

,0,1,2,3,4,5,6,7,8,9,...,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353
0,0.0,0.0,1.7,0.0,1.7,1.7,0.6,0.6,0.6,0.0,...,10.2,8.5,10.8,10.8,9.7,6.3,11.3,11.7,5.8,7.0
1,0.0,0.0,1.7,0.0,1.7,1.7,0.6,0.6,0.6,0.0,...,10.2,8.5,10.8,10.8,9.7,6.3,11.3,11.7,5.8,7.0
2,1.7,1.7,0.0,1.7,0.0,0.0,2.3,2.3,2.3,1.7,...,11.7,10.0,12.1,12.1,11.0,6.0,11.0,11.4,5.0,6.2
3,0.0,0.0,1.7,0.0,1.7,1.7,0.6,0.6,0.6,0.0,...,10.2,8.5,10.8,10.8,9.7,6.3,11.3,11.7,5.8,7.0
4,1.7,1.7,0.0,1.7,0.0,0.0,2.3,2.3,2.3,1.7,...,11.7,10.0,12.1,12.1,11.0,6.0,11.0,11.4,5.0,6.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,5.8,5.8,5.5,5.8,5.5,5.5,6.0,6.0,6.0,5.8,...,8.3,6.6,8.9,8.9,7.8,0.0,7.9,8.3,7.7,7.0
1350,11.7,11.7,11.4,11.7,11.4,11.4,12.3,12.3,12.3,11.7,...,6.7,8.4,12.4,12.4,11.3,8.8,0.0,0.4,13.5,12.8
1351,11.3,11.3,11.0,11.3,11.0,11.0,11.9,11.9,11.9,11.3,...,7.0,8.7,12.7,12.7,11.6,8.4,0.3,0.0,13.8,13.1
1352,8.4,8.4,6.7,8.4,6.7,6.7,7.9,7.9,7.9,8.4,...,10.5,8.8,9.8,9.8,8.7,9.7,17.3,17.0,0.0,3.9


In [ ]:
accessibility_matrix2 = pd.read_pickle("/content/ac_matrix/ac_matrix_20_10.pickle")
accessibility_matrix2

,0,1,2,3,4,5,6,7,8,9,...,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353
0,0.0,0.0,2.0,0.0,2.0,2.0,0.9,0.9,0.9,0.0,...,13.0,11.1,14.1,14.1,12.3,7.4,14.5,15.2,7.3,8.7
1,0.0,0.0,2.0,0.0,2.0,2.0,0.9,0.9,0.9,0.0,...,13.0,11.1,14.1,14.1,12.3,7.4,14.5,15.2,7.3,8.7
2,2.0,2.0,0.0,2.0,0.0,0.0,2.9,2.9,2.9,2.0,...,14.5,12.6,15.0,15.0,13.2,7.0,14.1,14.8,6.0,7.4
3,0.0,0.0,2.0,0.0,2.0,2.0,0.9,0.9,0.9,0.0,...,13.0,11.1,14.1,14.1,12.3,7.4,14.5,15.2,7.3,8.7
4,2.0,2.0,0.0,2.0,0.0,0.0,2.9,2.9,2.9,2.0,...,14.5,12.6,15.0,15.0,13.2,7.0,14.1,14.8,6.0,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,6.8,6.8,6.4,6.8,6.4,6.4,7.7,7.7,7.7,6.8,...,10.0,8.1,11.1,11.1,9.3,0.0,10.6,11.3,9.1,8.1
1350,14.7,14.7,14.3,14.7,14.3,14.3,15.6,15.6,15.6,14.7,...,8.0,10.0,15.5,15.5,13.7,11.2,0.0,0.7,15.7,14.7
1351,14.0,14.0,13.6,14.0,13.6,13.6,14.9,14.9,14.9,14.0,...,8.6,10.6,16.1,16.1,14.3,10.5,0.6,0.0,16.3,15.3
1352,10.1,10.1,8.1,10.1,8.1,8.1,10.1,10.1,10.1,10.1,...,12.5,10.6,12.2,12.2,10.4,11.5,21.0,20.4,0.0,4.6
